In [ ]:
 import pandas as pd

df1 = pd.read_csv(filepath_or_buffer = 'twitter_training.csv', names=['ID', 'USER', 'TYPE', 'TEXT'], na_values='Irrelevant')
df2 = pd.read_csv(filepath_or_buffer = 'twitter_validation.csv', names=['ID', 'USER', 'TYPE', 'TEXT'], na_values='Irrelevant')
df = pd.concat([df1, df2])
df.dropna(inplace=True) #delete irrelevant tweets
df['TYPE'] = df['TYPE'].map({'Positive': 1, 'Neutral': 0, 'Negative': -1})
df.drop_duplicates(inplace=True)
df.reset_index(drop = True, inplace = True)
df.info()
df['TEXT']

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59515 entries, 0 to 59514
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      59515 non-null  int64 
 1   USER    59515 non-null  object
 2   TYPE    59515 non-null  int64 
 3   TEXT    59515 non-null  object
dtypes: int64(2), object(2)
memory usage: 1.8+ MB


0        im getting on borderlands and i will murder yo...
1        I am coming to the borders and I will kill you...
2        im getting on borderlands and i will kill you ...
3        im coming on borderlands and i will murder you...
4        im getting on borderlands 2 and i will murder ...
                               ...                        
59510    I have noticed streamers I watch who are now p...
59511    @6th__man playing red dead redemption-\n\n“Oh ...
59512    ♥️ Suikoden 2\n1️⃣ Alex Kidd in Miracle World\...
59513    Thank you to Matching funds Home Depot RW paym...
59514    Late night stream with the boys! Come watch so...
Name: TEXT, Length: 59515, dtype: object

#Tweet preprocessor library

In [ ]:
# !pip install tweet-preprocessor
# -> Just deletes emojis -> not good

In [ ]:
# import preprocessor as p
# X = []
# for word in df['TEXT']:
#   X.append(p.clean(word))

#Handmade preprocessing

In [ ]:
!pip install emoji

import emoji
from string import punctuation
import re
import nltk
nltk.download('wordnet')
nltk.download('punkt')

from nltk import word_tokenize
from nltk.stem import PorterStemmer

stemmer = PorterStemmer()
text_lst = []
for text in df['TEXT']:
  newtext = ''
  for word in text.split(): #remove mentions and links
    if word[0]!='@':
      if '.' not in word or '/' not in word:
        newtext += word + ' '
  newtext = emoji.demojize(newtext) #transform emojis to text
  word_list = []
  for word in word_tokenize(newtext): #remove punctuation
    if word not in punctuation:
      word_list.append(word)
  stemmed_words = [stemmer.stem(word) for word in word_list]
  newtext = ''
  for word in stemmed_words:
    newtext += re.sub('[^a-zA-Z#]', '', word) + ' '
  text_lst.append(newtext)
y = df['TYPE']
#for i in range(10):
#print(X[i], y[i])

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 2.5 MB/s eta 0:00:00


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(min_df = 5, max_df = .7, stop_words='english', ngram_range=(1,1))
X = vectorizer.fit_transform(text_lst)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import svm
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = .8)
model = svm.SVC(kernel = 'linear')
model.fit(X_train, y_train)
model.score(X_test, y_test)

0.8027388053431908